### NMODL SymPy Visitor Examples

Some examples of the use of SymPy within NMODL to
- solve differential equations to generate solutions for the CNEXP solver (`SympySolverVisitor`)
- differentiate current expressions to generate CONDUCTANCE statements (`SympyConductanceVisitor`)

Please see the notebook `nmodl-python-tutorial.ipynb` for a more general tutorial on using the NMODL python interface, including installation instructions.

In [1]:
import nmodl.dsl as nmodl
from nmodl.dsl import visitor
from nmodl.dsl import ast
from nmodl.dsl import symtab

hh.mod input file:

In [2]:
channel = """
NEURON {
    SUFFIX hh
    USEION na READ ena WRITE ina
    USEION k READ ek WRITE ik
    NONSPECIFIC_CURRENT il
    RANGE gnabar, gkbar, gl, el, gna, gk
    RANGE minf, hinf, ninf, mtau, htau, ntau
}
 
UNITS {
    (mV) = (millivolt)
    (S) = (siemens)
}
 
PARAMETER {
    gnabar = .12 (S/cm2)
    gkbar = .036 (S/cm2)
    gl = .0003 (S/cm2)
    el = -54.3 (mV)
    celsius
}
 
STATE {
    m h n
}
 
ASSIGNED {
    v (mV)
 
    gna (S/cm2)
    gk (S/cm2)
    minf
    hinf
    ninf
    mtau (ms)
    htau (ms)
    ntau (ms)
}
 
BREAKPOINT {
    SOLVE states METHOD cnexp
    gna = gnabar*m*m*m*h
    ina = gna*(v - ena)
    gk = gkbar*n*n*n*n
    ik = gk*(v - ek)
    il = gl*(v - el)
}
 
INITIAL {
    rates(v, celsius)
    m = minf
    h = hinf
    n = ninf
}
 
DERIVATIVE states {
    rates(v, celsius)
    m' = (minf-m)/mtau
    h' = (hinf-h)/htau
    n' = (ninf-n)/ntau
}
 
PROCEDURE rates(v, celsius)
{
    LOCAL  alpha, beta, sum, q10
 
    q10 = 3^((celsius - 6.3)/10)
 
    :"m" sodium activation system
    alpha = .1 * vtrap(-(v+40),10)
    beta =  4 * exp(-(v+65)/18)
    sum = alpha + beta
    mtau = 1/(q10*sum)
    minf = alpha/sum
 
    :"h" sodium inactivation system
    alpha = .07 * exp(-(v+65)/20)
    beta = 1 / (exp(-(v+35)/10) + 1)
    sum = alpha + beta
    htau = 1/(q10*sum)
    hinf = alpha/sum
 
    :"n" potassium activation system
    alpha = .01*vtrap(-(v+55),10)
    beta = .125*exp(-(v+65)/80)
    sum = alpha + beta
    ntau = 1/(q10*sum)
    ninf = alpha/sum
}
 
FUNCTION vtrap(x,y) {
    : use built in exprelr(z) = z/(exp(z)-1), which handles the z=0 case correctly
    vtrap = y*exprelr(x/y)
}
"""

### ODE Solver example

In [3]:
# parse NMDOL file
driver = nmodl.Driver()
driver.parse_string(channel)
modast = driver.ast()
lookup_visitor = visitor.AstLookupVisitor()

In [4]:
# print solve method
print(nmodl.to_nmodl(lookup_visitor.lookup(modast, ast.AstNodeType.SOLVE_BLOCK)[0]))

SOLVE states METHOD cnexp


In [5]:
# print DERIVATIVE block
print(nmodl.to_nmodl(lookup_visitor.lookup(modast, ast.AstNodeType.DERIVATIVE_BLOCK)[0]))

DERIVATIVE states {
    rates(v, celsius)
    m' = (minf-m)/mtau
    h' = (hinf-h)/htau
    n' = (ninf-n)/ntau
}


Run `SympySolverVisitor`, then print DERIVATIVE block again:

In [6]:
# first need to run SymtabVisitor to generate Symbol Table
symv = symtab.SymtabVisitor()
symv.visit_program(modast)
# then we can run SympySolverVisitor
sympy_solver_visitor = visitor.SympySolverVisitor()
sympy_solver_visitor.visit_program(modast)
# print DERIVATIVE block
print(nmodl.to_nmodl(lookup_visitor.lookup(modast, ast.AstNodeType.DERIVATIVE_BLOCK)[0]))

DERIVATIVE states {
    rates(v, celsius)
    m = minf+(m-minf)*exp(-dt/mtau)
    h = hinf+(h-hinf)*exp(-dt/htau)
    n = ninf+(n-ninf)*exp(-dt/ntau)
}


Repeat the process, but this time with the option `use_pade_approx` set to `True`.

This returns the (1,1) Pade approximant to the analytic solution, which is correct to second order in `dt`

In [7]:
# parse NMDOL file
driver = nmodl.Driver()
driver.parse_string(channel)
modast = driver.ast()
lookup_visitor = visitor.AstLookupVisitor()
# first need to run SymtabVisitor to generate Symbol Table
symv = symtab.SymtabVisitor()
symv.visit_program(modast)
# then we can run SympySolverVisitor
sympy_solver_visitor = visitor.SympySolverVisitor(use_pade_approx=True)
sympy_solver_visitor.visit_program(modast)
# print DERIVATIVE block
print(nmodl.to_nmodl(lookup_visitor.lookup(modast, ast.AstNodeType.DERIVATIVE_BLOCK)[0]))

DERIVATIVE states {
    rates(v, celsius)
    m = (-dt*m+2*dt*minf+2*m*mtau)/(dt+2*mtau)
    h = (-dt*h+2*dt*hinf+2*h*htau)/(dt+2*htau)
    n = (-dt*n+2*dt*ninf+2*n*ntau)/(dt+2*ntau)
}


### CONDUCTANCE example

In [8]:
# parse NMDOL file
driver = nmodl.Driver()
driver.parse_string(channel)
modast = driver.ast()
lookup_visitor = visitor.AstLookupVisitor()

In [9]:
# print USEION and NONSPECIFIC current statements
for node in lookup_visitor.lookup(modast, ast.AstNodeType.USEION or ast.AstNodeType.NONSPECIFIC):
    print(nmodl.to_nmodl(node))

USEION na READ ena WRITE ina
USEION k READ ek WRITE ik


In [10]:
# print BREAKPOINT
print(nmodl.to_nmodl(lookup_visitor.lookup(modast, ast.AstNodeType.BREAKPOINT_BLOCK)[0]))

BREAKPOINT {
    SOLVE states METHOD cnexp
    gna = gnabar*m*m*m*h
    ina = gna*(v-ena)
    gk = gkbar*n*n*n*n
    ik = gk*(v-ek)
    il = gl*(v-el)
}


BREAKPOINT block after running `SympyConductanceVisitor`:

In [11]:
# first need to run SymtabVisitor to generate Symbol Table
symv = symtab.SymtabVisitor()
symv.visit_program(modast)
# then we can run SympyConductanceVisitor
sympy_conductance_visitor = visitor.SympyConductanceVisitor()
sympy_conductance_visitor.visit_program(modast)
# print BREAKPOINT block
print(nmodl.to_nmodl(lookup_visitor.lookup(modast, ast.AstNodeType.BREAKPOINT_BLOCK)[0]))

BREAKPOINT {
    CONDUCTANCE gna USEION na
    CONDUCTANCE gl
    CONDUCTANCE gk USEION k
    SOLVE states METHOD cnexp
    gna = gnabar*m*m*m*h
    ina = gna*(v-ena)
    gk = gkbar*n*n*n*n
    ik = gk*(v-ek)
    il = gl*(v-el)
}
